In [1]:
import sys
import os

functionPath = os.path.join( '../../', 'text_mining' )
sys.path.append( functionPath )

import get_vectors as gv

In [2]:
import numpy as np
import pandas as pd 
from sklearn.svm import SVC
from sklearn import metrics 

LABEL_CNT = 3 

In [21]:
SVD_vec = gv.get_SVD_vectors( '../../text_mining/', '../../src_txt/', 100 )

In [ ]:
LDA_vec = gv.get_LDA_vectors( '../../text_mining/', '../../src_txt/', 100 )
TFIDF_vec, _ = gv.get_tf_idf( '../../text_mining/', '../../src_txt/' )
TFIDF_vec = TFIDF_vec.toarray()

In [22]:
df = pd.read_csv( '../../text_mining/semi_super_label.csv')
train_index = sorted( list( set( df['file'] ) ) )

In [23]:
train_y = []
for i in range( len( train_index ) ):
    train_y.append( [ False ] * LABEL_CNT )
    labels = df[ 'labels' ][ i ].rstrip()[1:-1].split(' ')
    if labels[0] != '' :
        for label in labels : 
            l = int( label ) - 1 
            train_y[-1][l] = True  
    train_y[-1] = np.array( train_y[-1] )
train_y = np.array( train_y )

In [24]:
train_x = [SVD_vec[i] for i in train_index]

In [25]:
SVM_model = SVC( kernel='poly', degree=2, coef0=1, C=1 )

In [28]:
from sklearn.model_selection import cross_val_score

score = cross_val_score( SVM_model, train_x, train_y[:,2], cv=10, scoring='f1_macro' )
print( score.mean() )

0.8417930260674261


In [7]:
SVM_model_1 = SVC( kernel='poly', degree=2, coef0=1, C=1 )
SVM_model_2 = SVC( kernel='poly', degree=2, coef0=1, C=1 )
SVM_model_3 = SVC( kernel='poly', degree=2, coef0=1, C=1 )

In [29]:
SVM_model_1.fit( train_x, train_y[ :, 0 ] )
SVM_model_2.fit( train_x, train_y[ :, 1 ] )
SVM_model_3.fit( train_x, train_y[ :, 2 ] )

SVC(C=1, coef0=1, degree=2, kernel='poly')

In [30]:
predict_y = []
predict_y.append( SVM_model_1.predict( train_x ) )
predict_y.append( SVM_model_2.predict( train_x ) )
predict_y.append( SVM_model_3.predict( train_x ) )

In [38]:
for i in range( 3 ):
    result = predict_y[ i ]
    with open( f'../../tm_web/backend/data/type{i+1}.csv', 'w' ) as f:
        f.write( f'{i}' )
        for i in range( len( result ) ):
            if result[ i ] == True:
                f.write( f'\n{i}.txt' )